# 🧠 HealthAI: Intelligent Healthcare Assistant in Google Colab

In [ ]:
!pip install transformers gradio plotly pandas scikit-learn --quiet

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "ibm-granite/granite-3.3-2b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

health_bot = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:

def patient_chat(query):
    try:
        prompt = f"You are a kind and knowledgeable virtual medical assistant.\nUser: {query}\nAssistant:"
        result = health_bot(prompt, max_new_tokens=200, temperature=0.7)
        return result[0]['generated_text']
    except Exception as e:
        return f"❌ Chat Error: {str(e)}"


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/symptoms_disease_dataset.csv")

# Split the 'Symptoms' column into individual symptom strings
df['Symptoms'] = df['Symptoms'].str.split(', ')

# Create a list of all unique symptoms
all_symptoms = sorted(list(set([item for sublist in df['Symptoms'] for item in sublist])))

# Create new columns for each symptom, with 1 if present and 0 if not
for symptom in all_symptoms:
    df[symptom] = df['Symptoms'].apply(lambda x: 1 if symptom in x else 0)

# Now X will contain the one-hot encoded symptom columns, and y remains the 'Disease' column
X = df[all_symptoms]
y = df["Disease"]

# Check the first few rows of the processed X to verify the format
print(X.head())

clf = RandomForestClassifier()
clf.fit(X, y)


   Cough  Fatigue  Fever  Headache  Nausea  Shortness of Breath  Sore Throat
0      0        1      0         1       1                    1            1
1      0        0      0         1       1                    1            1
2      0        0      0         0       0                    0            1
3      1        0      0         0       0                    0            0
4      0        0      0         0       0                    0            1


RandomForestClassifier()

In [ ]:

def predict_disease(symptom_text):
    try:
        symptom_text = symptom_text.lower()
        symptoms = [1 if sym in symptom_text else 0 for sym in X.columns]
        return clf.predict([symptoms])[0]
    except Exception as e:
        return f"❌ Prediction Error: {str(e)}"


In [ ]:

def treatment_plan(disease, age=30, allergies="None", history="None"):
    try:
        prompt = (
            f"You are a doctor. Provide a treatment plan for a patient with:\n"
            f"- Disease: {disease}\n"
            f"- Age: {age if age > 0 else 30}\n"
            f"- Allergies: {allergies or 'None'}\n"
            f"- Medical History: {history or 'None'}\n"
            f"Include medications, lifestyle changes, and follow-ups."
        )
        result = health_bot(prompt, max_new_tokens=300)
        return result[0]['generated_text']
    except Exception as e:
        return f"❌ Treatment Error: {str(e)}"


In [ ]:

import plotly.express as px

def analytics_dashboard(file_obj):
    try:
        df = pd.read_csv(file_obj)
        fig = px.line(df, x="Date", y=["HeartRate", "BP_Systolic", "BloodSugar"],
                      title="📊 Health Analytics Dashboard",
                      labels={"value": "Measurement", "variable": "Metric"})
        return fig
    except Exception as e:
        return f"Error processing file: {str(e)}"


In [ ]:

import gradio as gr

def health_ai(query, symptoms, age, allergies, history):
    chat = patient_chat(query)
    diagnosis = predict_disease(symptoms)
    plan = treatment_plan(diagnosis, age, allergies, history)
    return chat, diagnosis, plan

with gr.Blocks(title="🧠 HealthAI: Intelligent Healthcare Assistant") as demo:
    gr.Markdown("# 🧠 HealthAI")
    gr.Markdown("Ask health questions, predict diseases, receive treatment plans, and view your health metrics.")

    with gr.Tabs():
        with gr.TabItem("🧬 Health Assistant"):
            with gr.Row():
                query = gr.Textbox(label="🗨️ Health Question")
                symptoms = gr.Textbox(label="🧬 Symptoms (comma-separated)")
            with gr.Row():
                age = gr.Number(label="🎂 Age")
                allergies = gr.Textbox(label="⚠️ Allergies")
                history = gr.Textbox(label="📖 Medical History")
            btn = gr.Button("Run HealthAI")
            chat_output = gr.Textbox(label="💬 Assistant Response")
            prediction_output = gr.Textbox(label="🔍 Predicted Disease")
            plan_output = gr.Textbox(label="💊 Treatment Plan")
            btn.click(health_ai, inputs=[query, symptoms, age, allergies, history],
                      outputs=[chat_output, prediction_output, plan_output])

        with gr.TabItem("📈 Health Analytics"):
            file_input = gr.File(label="📂 Upload Health Data CSV", file_types=[".csv"])
            graph_output = gr.Plot(label="📊 Health Trends")
            file_input.change(fn=analytics_dashboard, inputs=file_input, outputs=graph_output)

    gr.Markdown("⚠️ This tool does not provide medical advice. Always consult a healthcare provider.")

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a809d10a26e8ae416a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
